<a href="https://colab.research.google.com/github/Thukyd/TradeRepublic_Portfolio/blob/master/TradeRepublic_PDF_Converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0.3 | TradeRepublic Portfolio** 
# *PDF Converter for Portfolio Performance & Investing.com*
# a) Description
This tool should help to keep the overview of the transactions within TradeRepublic. In TR there is the possibility to export all orders as PDF. For testing purposes the import and export is currently only possible with G-Drive. Alternatives are on the todo list. 


## Output A: Master Sheet
The master sheet lists all previous transactions from the PDFs. 

## Output B: Delta Sheet (dated)
The delta sheet contains all new transactions that have been added since the script was last executed. These are intended to be imported into Portfolio Performance and Investing.com. 

## Scope 
*   optimized for Google Colab (https://colab.research.google.com/)
*   export optimized for "Portfolio Performance" (https://www.portfolio-performance.info/) and "Investing.com" (https://de.investing.com/)
*   For further information about the import at "Investing.com", see: https://www.investing-support.com/hc/en-us/articles/360000265217-Import-Portfolio-Holdings 


# b) Configuration
## G-Drive Path
*   TradeRepublic statements will be imported via GoogleDrive. All PDF files shoule be in a single folder. You need to configure the path to your G-Drive before usage (see: "Configurations - to be defined by user")

## Finnhub Token
* If you want to add a ticker symbol to the sheets you need to have a Finnhub API token. You can register @ https://finnhub.io/ and have to paste your API key in the configs. 
* You can ignore this, if you don't want to register for a key. 


# c) Options to customize
- for different data sources, see: https://colab.research.google.com/notebooks/io.ipynb
- in order to create different data structures, take a look at "Examples for extracted fields". 

# d) Further Information
## Handling of costs statements
*   "Kosten des Wertpapierkaufs/verkaufs" are be considered.
*   "Kosten während der Haltedauer (pro Jahr)" are not extracted and therefore do not appear the sheets. 

## Deposits & Withdrawls
* deposits and withdrawals to the depot are not recorded in PDF format. Therefore they are not taken into account and must be entered by hand if necessary.

## "Order" in Trade Republic documents
* each TradeRepublic document has got a "order" number. This is extracted and stored in the field "Notiz". It serves to prevent duplicate entries.  

# e) Backlog
## Open
* create sheets for portfolio performance in .csv format
* offer alternatives to G-Drive import/export
* Investing.com => Import change "Kauf" and "Verkauf" into "Kaufen" and "Verkaufen"

## To be fixed
- define "add to delta" by order number - not by row alone
- limit reading operations for google sheets https://github.com/burnash/gspread/issues/583
- examples

# f) Changelog 
## 0.1
* extract G-Drive folder of TradeRepublic PDFs
* create data structure (for Portfolio Performance or other purposes)
* generate master sheet of all transactions
* generate delta sheet for new transactions (base for TradeRepublic import)

## 0.2
- fixed | sort extracted transactions by date
- fixed | double entries of table lables in delta sheet

## 0.3
- add ticker symbol via finnhub api (requires your own api key)
- fixed | calculate purchase price
- add values to sheet ("Kaufkurs", "Gebühren")
- data structure for Investing.com import


# Configurations - to be defined by user

In [ ]:
# GDRIVE
# path to gdrive folder with your TradeRepublic pdfs
path_gdrive = "/content/drive/My Drive/YOUR_FOLDER/"

# FINNHUB - Ticker search (optional) 
  # request is used to get receive the ticker 
  # in order to use this, you need to register @ https://finnhub.io/ - the free version is sufficient. 
  # fill in your api token here: 
finnhub_token = "YOUR_FINNHUB_API_KEY"

# Imports

In [ ]:
# PyPDF2 for PDF extraction
!pip install PyPDF2
import PyPDF2
# Regex for text processing
import re
# requirement for pdf-folder extraction
import glob
import os
# requirement for get_date_today(), convert_str_to_date()
from datetime import date, datetime
# optional - used for finhub ticker search
import requests

# Extraction & helper functions

In [ ]:
def get_date_today ():
  today = date.today()
  return today.strftime("%Y-%m-%d")

In [ ]:
def convert_to_float(input):
  decimal_point = input.replace(",", ".") # input: 20.000,00 ; output: 20.000.00
  result = decimal_point.replace(".", "", decimal_point.count(".") -1) # input: 20.000.00 ; output: 20000.00
  return float(result)

In [ ]:
def convert_str_to_date(input_string):
  output = datetime.strptime(input_string, "%d.%m.%Y").date()
  return output

In [ ]:
def convert_date_to_str(input_date):
  output = input_date.strftime("%d.%m.%Y")
  return output

In [ ]:
def get_ticker(isin): # OPTIONAL - requires Finnhub API token
  query = finnhub_token + "&isin=" + isin
  try:
    r = requests.get("https://finnhub.io/api/v1/stock/profile2?token=" + query)
    finn_response = r.json()
    return finn_response["ticker"]
  except: # if the ticker wasn't found or the user didn't specify the api key
    return

In [ ]:
def preprocess_letter (full_extraction):
  header = "WERTPAPIERGESCHÄFTWERTPAPIERORDER"   # relevant pdf
  header, core = full_extraction.split(header)
  return header, core

In [ ]:
def extract_transaction_date (raw_text):
  pattern = "DATUM\d{2}.\d{2}.\d{4}"
  transaction_date = re.findall(pattern, raw_text)
  date = re.split("DATUM", transaction_date[0])
  return date[1]

In [ ]:
def extract_order_id (raw_text):
  pattern = "(?<=ORDER)(.*?)(?=DEPOT)"
  id = re.findall(pattern, raw_text)
  return id[0]

In [ ]:
def extract_isin(raw_text):
  # Search for ISIN of Stock
  isin_pattern = "(?<=ISIN: )(BE|BM|FR|BG|VE|DK|HR|DE|JP|HU|HK|JO|BR|XS|FI|GR|IS|RU|LB|PT|NO|TW|UA|TR|LK|LV|LU|TH|NL|PK|PH|RO|EG|PL|AA|CH|CN|CL|EE|CA|IR|IT|ZA|CZ|CY|AR|AU|AT|IN|CS|CR|IE|ID|ES|PE|TN|PA|SG|IL|US|MX|SK|KR|SI|KW|MY|MO|SE|GB|GG|KY|JE|VG|NG|SA|MU)([0-9A-Z]{9})([0-9])"
  matches = re.findall(isin_pattern, text)
  item = "".join(matches[0])
  return item

In [ ]:
def extract_name(raw_text):
  pattern = "(?<=ANZAHLWERTAUSFÜHRUNGSPLATZ)(.*?)(?=ISIN)"
  name = re.findall(pattern, raw_text)
  return name[0]

In [ ]:
def extract_order(raw_text):
  ISIN = extract_isin(raw_text)
  pattern = "(?<="+ ISIN +")(.*?)(?=Lang und Schwarz Exchange)"
  order = re.findall(pattern, raw_text)
  matched = ''.join(order)
  matched = matched.split(" ")
  transaction_type = matched[0]
  order_count, total_value = matched[2].split("Stk.") 
  return [transaction_type, order_count, total_value]

In [ ]:
# investing.com needs "Buy" & "Sell" as input instead of "Kauf" & "Verkauf"
def investing_transaction_type(input):
  if input == "Kauf":
    input = "Buy"
  if input == "Verkauf":
    input = "Sell"
  return input

In [ ]:
# does not include running costs
# takes first appearence of costs in the pdf
  # buy operations => consists of buying costs, running costs & selling costs | only buying costs considered
  # sell operation => consits of selling costs | selling costs considered
def extract_order_costs(raw_text):
  transaction_type = extract_order(raw_text)[0]
  pattern = "(?<=SUMME)(.*?)(?= )"
  costs = re.findall(pattern, raw_text)
  return costs[0]

In [ ]:
# In TradeRepublic statements all total values for buy & sell are in positive numbers.
# For the conversion to Portfolio Performance (PP) this value has got to take the type of transaction in account.
# It needs to have a sign. Negative numbers for "buy" and positive numbers for "sell".
def add_sign_to_order_value(transaction_type, total_value):
  if transaction_type == "Kaufen":
    return "-" + total_value
  else: 
    return total_value

In [ ]:
def calculate_stock_price(count, total_price):
  # transform string to float
  float_price = convert_to_float(total_price)
  float_count = float(count)
  # calculate price of single stock
  calculated = float_price / float_count
  # convert calculated float into string again
  german_calculated = convert_flot_to_german_decimal(calculated)
  return german_calculated

In [ ]:
def convert_flot_to_german_decimal(number):
  # build format string
  format_str = "{{:,.{}f}}".format(2)
  # make number string
  number_str = format_str.format(number)
  # replace chars
  return number_str.replace(',', 'X').replace('.', ',').replace('X', '.')

# https://stackoverflow.com/questions/55616520/is-there-a-simple-and-preferred-way-of-german-number-string-formatting-in-python/55616843

# Data Structure
- optimized for Portfolio Performance
- for different data sources, see: https://colab.research.google.com/notebooks/io.ipynb


## a) extract pdf files

In [ ]:
# Load Google Drive helper
from google.colab import drive
# This will prompt for authorization
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# create list to save extracted data
transaction_record = []
for filename in glob.glob(os.path.join(path_gdrive, '*.pdf')):
   with open(filename, 'rb') as fin: # open in readonly mode
      # read and extract pdf infos
      pdf_reader = PyPDF2.PdfFileReader(fin)
      # extract first page
      extr_page = pdf_reader.getPage(0)
      text = extr_page.extractText()
      # check if current pdf file is a "WERTPAPIERGESCHÄFTWERTPAPIERORDER"
      transaction_pdf = "WERTPAPIERGESCHÄFTWERTPAPIERORDER"
      duplicate_file = ").pdf"

      if duplicate_file in filename: # try to find pdf duplicates - eg. "filename (1).pdf" instead of "filename.pdf"
          #print("Processing PDF | DUPLICATE   | \"Wertpapiergeschäftsorder\" |  " + filename)
          continue
      elif transaction_pdf in text:  ## all non duplicate,"Wertpapergeschäftsorder"
          #print("Processing PDF | RELEVANT    | \"Wertpapiergeschäftsorder\" |  " + filename)
          pass
      else: # non duplicate, irelevant files
          #print("Processing PDF | IRELEVANT   |                            |  " + filename)
          continue

      # split document for easier processing
      full_text = preprocess_letter(text)
      text = full_text[1]
      header = full_text[0]

      # get order information
      transaction_type, order_count, total_value = extract_order(text)
      price = calculate_stock_price(order_count, total_value)
      order_value_with_sign = add_sign_to_order_value(transaction_type, total_value)

      # set data structure
      current_transaction = {
      "Datum" : extract_transaction_date(header),
      "ISIN" : extract_isin(text),
      "Typ": transaction_type,
      "Art": investing_transaction_type(transaction_type),
      "Wert" : order_value_with_sign,
      "Buchungswährung" : "EUR",
      "Steuern" : "",
      "Stück" : order_count,
      "Werpapiername" : extract_name(text),
      "Notiz" : extract_order_id(header),
      "Gebühren" : extract_order_costs(text),
      "WKN" : "",
      "Ticker-Symbol": get_ticker(extract_isin(text)),
      "Kaufkurs" : price
      }

      transaction_record.append(current_transaction)

# https://stackoverflow.com/questions/72899/how-do-i-sort-a-list-of-dictionaries-by-a-value-of-the-dictionary
record = sorted(transaction_record, key=lambda k: convert_str_to_date(k["Datum"])) 
record

## b) create structure for Portfolio Performance

In [ ]:
# prepare transaction record for spreadsheet
data = []
# portfolio_performance scheme for .csv import
pp_table_labels = ["Datum", "Typ", "Art" , "Wert", "Kaufkurs", "Stück", "Wertpapiername", "ISIN", "WKN", "Ticker-Symbol", "Gebühren", "Steuern", "Buchungswährung", "Notiz"]

for transaction in range(len(record)):
  data.append([
                        record[transaction]["Datum"],
                        record[transaction]["Typ"],
                        record[transaction]["Art"],
                        record[transaction]["Wert"],
                        record[transaction]["Kaufkurs"],
                        record[transaction]["Stück"],
                        record[transaction]["Werpapiername"],
                        record[transaction]["ISIN"],
                        record[transaction]["WKN"],
                        record[transaction]["Ticker-Symbol"],
                        record[transaction]["Gebühren"],
                        record[transaction]["Steuern"],
                        record[transaction]["Buchungswährung"],
                        record[transaction]["Notiz"]
                        ])
data

# Update Master & Delta Sheet

In [ ]:
#@title
!pip install --upgrade --quiet gspread

In [ ]:
#@title
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
#@title
# Delta Sheet
  # creates/updates a list of new transactions 
  # will be called in update_master_sheets
def update_delta_sheet(data_row, labels):
  today = get_date_today ()

  ## open or create delta sheet
  try: # Check if Spreadsheet already exists
    spreadsheet_delta = gc.open("TradeRepublic_Delta_" + today) # open existing sheet
    worksheet_delta = spreadsheet_delta.sheet1
    print("   Delta | Open")
  except:
    spreadsheet_delta = gc.create("TradeRepublic_Delta_" + today) # create new sheet
    worksheet_delta = spreadsheet_delta.sheet1
    print("   Delta | Create")
    
  if worksheet_delta.acell("A1").value == "": # add labeling if missing
    worksheet_delta.update("A1", [labels])

  # don't overwrite existing entries, but check for the first free row before inserting current item
  count = 0
  while True:
    count += 1
    if worksheet_delta.acell("A" + str(count)).value == "":
      worksheet_delta.update("A" + str(count), [data_row])
      print("   Delta | Entry added to delta")      
      break

In [ ]:
# Master Sheet
  # creates/updates full record sheet 
  # calls delta sheet to note down new entries
def update_master_sheet(worksheet_data, labels):
  try: # Check if Spreadsheet already exists
    spreadsheet = gc.open("TradeRepublic_Master") # open existing sheet
    print("Master | Open sheet\n")
  except:
    spreadsheet = gc.create("TradeRepublic_Master") # create new sheet
    print("Master | Create sheet\n")
  worksheet_master = spreadsheet.sheet1 # get worksheet for document
  worksheet_master.update("A1", [labels]) # add table label to document

  last_column = len(worksheet_data[0]) # check for last column ("Notiz")

  # Update information by order id
  for row in range(len(worksheet_data)): 
    current_data_item = worksheet_data[row][last_column-1] # data from extraction; "last coulmn" is the index column for "Notiz". In Notiz there is the TradeRepublic Order Id
    current_sheet_row = worksheet_master.acell("K" + str(row+2)).value # equivalent data in sheets

    if current_sheet_row == current_data_item:   # update only new entries (defined by order_id & row in sheet)
      print("Master | Entry exists already")
    else:
      print("Master | Entry added to master")
      worksheet_master.update("A" + str(row+2), [worksheet_data[row]]) # update sheet row by row
      update_delta_sheet(worksheet_data[row], labels)

In [ ]:
# Execute sheet update
update_master_sheet(data, pp_table_labels)

# Examples for extracted values

In [ ]:
# extract ISNIN
ISIN = extract_isin(text)
ISIN

In [ ]:
# date of transaction
date = extract_transaction_date(header)
date

In [ ]:
# FIXME: 
# name of position
# name = extract_name(text)
# name

In [ ]:
# FIXME: 
# transaction type & order volume
# transaction_type, order_count, total_value = extract_order(text)
# transaction_type

In [ ]:
# FIXME: 
#  costs = extract_order_costs(text)
#  costs

In [ ]:
# TODO: needs to be fixed
# stock_value = calculate_stock_value(text)
# stock_value

In [ ]:
# TODO: needs to be fixed
# # add sign in total number for buy/sell actions
# order_value = add_sign_to_order_value(text)
# order_value

In [ ]:
# # Range of extraced fields
# print("Date : " + date)
# print("Werpapiername : " + name)
# print("ISIN : " + ISIN)
# print("Transaction Type : " + transaction_type)
# print("Order Count : " + order_count)
# #print("Stock Value : " + stock_value)
# print("Total Value : " + total_value)
# print("Transaction Costs : " + costs)
# print("Total Value with Sign: " + order_value)